In [1]:
import requests, praw, re, time
from bs4 import BeautifulSoup 
import pandas as pd 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import indeed_query,r_client_id,r_secret_key,r_dev,r_username,r_pw

In [5]:
reddit = praw.Reddit(
    client_id=r_client_id,
    client_secret=r_secret_key,
    password=r_pw,
    user_agent=f'testscript by u/{r_dev}',
    username=r_dev,
)
print(reddit.user.me())

Jazzy_Danger_1230


In [4]:
for submission in reddit.subreddit("wallstreetbets").hot(limit=25):
    print(submission.title)

Daily Discussion Thread for June 02, 2021
Most Anticipated Earnings Releases for the week beginning May 31st, 2021
We can’t stop... They can’t stop us, although they try... we’re true soldiers, we don’t die... We keep rolling... na-nah-nah-nah-nah!!!
The second coming
I put 750 dollars in I have never had this much I know my dads looking down smiling thankyou apes
BB to the MOON, leggooo let's get this money!!
I’m the person that Yolo’d my dads savings into BB 6 months ago, it’s either retirement on a yacht or foodstamps for him, the plan has NOT changed!
Can we just go ahead and take BB to the moon again?? 🚀🚀🧨
BB yolo, I’m still holding my 6/18
$32K In A Day - Theta Play - This Is The Way
I'd buy GME for a dollar!
Holding BB to the moon! 🚀
$BB🚀🚀 $37,000 yolo
What are gains on AMC so far? 🚀
Why AMC is rallying and why it will CONTINUE rallying 🚀🌑
Moon 🌙 is coming HOLD 💎🙌🏻
Ape like popcorn
When I finally sell at $100,000 a share
I heard BB goes brrrrr
96K Yolo on BB from 3/12. Where my 

In [ ]:
p=0
url = f'https://washingtondc.craigslist.org/d/bicycles/search/bia?s={p}'
soup = BeautifulSoup(requests.get(url).text, "html.parser")

   
    

In [ ]:
 els = soup.select('.jobsearch-SerpJobCard') 